---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [61]:
def blight_model():
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.metrics import roc_curve, auc

    #def blight_model():

    #%matplotlib notebook
    #import matplotlib.pyplot as plt
    #import seaborn as sns
    fields = ['ticket_id','judgment_amount','ticket_issued_date','hearing_date','disposition','compliance']
    df_train = pd.read_csv('train.csv', encoding = 'ISO-8859-1', usecols=fields ).fillna(0) 
    df_addresses = pd.read_csv('addresses.csv', encoding = 'ISO-8859-1' ).fillna(0) 
    df_latlons = pd.read_csv('latlons.csv', encoding = 'ISO-8859-1').fillna(0)

    df_addresses.head()
    df_latlons.head()
    df_all_locs=pd.merge(df_addresses, df_latlons, on='address')
    df_all_locs
    df_all_locs.set_index(['ticket_id'])
    
    df_train1=pd.merge(df_train,df_all_locs)
    df_train1 = df_train1.set_index(['ticket_id'])
    df_train1['date_diff'] = (pd.to_datetime(df_train1['hearing_date']).dt.date - pd.to_datetime(df_train1['ticket_issued_date']).dt.date).astype('timedelta64[D]').fillna(0)
    df_train1['date_diff'] = (pd.to_datetime(df_train1['hearing_date']).dt.date-pd.to_datetime(df_train1['ticket_issued_date']).dt.date).fillna(pd.Timedelta('-1 days')).dt.days
    del df_train1['ticket_issued_date']
    del df_train1['hearing_date']
    del df_train1['address']
    df_train3 = pd.get_dummies(df_train1)
    #df_train3


    test_fields = ['ticket_id','judgment_amount','ticket_issued_date','hearing_date','disposition']
    df_test = pd.read_csv('test.csv', encoding = 'ISO-8859-1', usecols=test_fields )#.dropna(axis=0, how='any')
    df_test1=pd.merge(df_test,df_all_locs,how='left')
    df_test1 = df_test1.set_index(['ticket_id'])
    df_test1['date_diff'] = (pd.to_datetime(df_test1['hearing_date']).dt.date - pd.to_datetime(df_test1['ticket_issued_date']).dt.date).astype('timedelta64[D]').fillna(0)
    df_test1['date_diff'] = (pd.to_datetime(df_test1['hearing_date']).dt.date-pd.to_datetime(df_test1['ticket_issued_date']).dt.date).fillna(pd.Timedelta('-1 days')).dt.days
    del df_test1['ticket_issued_date']
    del df_test1['hearing_date']
    del df_test1['address']
  
    df_test1=df_test1.fillna(0)
    df_test2 = pd.get_dummies(df_test1)
    
    X_tst=df_test2[['disposition_Responsible by Admission','disposition_Responsible by Determination',\
                   'date_diff','judgment_amount']]



    # Limiting features
    X = df_train3[['disposition_Responsible by Admission','disposition_Responsible by Determination',\
                   'date_diff','judgment_amount']]
    y = df_train3['compliance']

    # default is 75% / 25% train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)


    clf = DecisionTreeClassifier(max_depth = 3,random_state=0).fit(X_train, y_train)
    series = pd.Series(data=clf.feature_importances_, index=list(X_train)).sort_values(ascending=False)  
    featlist=series.iloc[0:4].index.tolist()
    featlist

    #['compliance', 'date_diff', 'judgment_amount', 'clean_up_cost', 'discount_amount']


    clf = GradientBoostingClassifier(max_depth = 3, random_state = 0).fit(X_train, y_train)
    y_score_clf = clf.predict(X_tst)
    #y_score_clf=clf.fit(X_train, y_train).decision_function(X_test)
    #y_proba_lr = clf.fit(X_test, y_test).predict_proba(X_tst)
    y_proba_lr = clf.fit(X_test, y_test).predict_proba(X_tst)
    
    
    #fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score_clf)
    #roc_auc_lr = auc(fpr_lr, tpr_lr)
   
    
    df_test2.insert(4, "compliance", y_proba_lr[0:,1]) #may need to be zero
    
    #return len(y_proba_lr[0:,1])
    return pd.to_numeric(df_test2["compliance"], downcast="float")
    #return df_train3
    #return roc_auc_lr
    #return featlist
    #return y_proba_lr
    #return list(df_test2)
blight_model()

ticket_id
284932    0.944510
285362    0.982621
285361    0.930596
285338    0.940262
285346    0.930596
285345    0.940262
285347    0.933200
285342    0.284503
285530    0.982621
284989    0.981979
285344    0.933200
285343    0.982621
285340    0.982621
285341    0.933200
285349    0.930596
285348    0.940262
284991    0.981979
285532    0.981979
285406    0.981979
285001    0.981979
285006    0.982621
285405    0.982621
285337    0.981979
285496    0.933200
285497    0.940262
285378    0.982621
285589    0.981979
285585    0.940262
285501    0.930596
285581    0.982621
            ...   
376367    0.981979
376366    0.961636
376362    0.741087
376363    0.679327
376365    0.981979
376364    0.961636
376228    0.966014
376265    0.966014
376286    0.875778
376320    0.961636
376314    0.961636
376327    0.737861
376385    0.737861
376435    0.815276
376370    0.076706
376434    0.933200
376459    0.930596
376478    0.991472
376473    0.961636
376484    0.977190
376482    0.981979
37